언어 판별기 딥러닝



검증까지 ......

#### [1] 모듈로딩 <hr>

In [189]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F 
from torch.utils.data import Dataset, DataLoader 
import torch.optim as optim    
from torchmetrics.classification import MulticlassF1Score
from torchinfo import summary 
import math
from collections import *

# 파일 관련
import os

#- Data 및 시각화 관련
import pandas as pd 
import matplotlib.pyplot as plt              
from sklearn.preprocessing import * 
from sklearn.model_selection import train_test_split

언어별로 사용하는 알파벳의 빈도가 다르다. 그 빈도수를 통해 딥러닝을 시켜서 어떤 언어인지 파악하도록 한다.

.....

split

In [190]:
TRAIN_PATH = r'C:\Git\KDT\BigData\data\lang\train\\'
TEST_PATH = r'C:\Git\KDT\BigData\data\lang\test\\'

files = os.listdir(TRAIN_PATH)
featureDF = pd.DataFrame()
targetSR = []

for file in files:
    # print(file, TEST_PATH+file, os.path.exists(TRAIN_PATH+file))
    label = file[:2]
    # print(f'Label => {label}')


    with open(TRAIN_PATH+file, mode = 'r') as f:
        alldata = f.read()
        alldata = alldata.lower()

        alldata = alldata.replace('\n','')

        for ch in alldata:
            if ch.isalpha():
                pass
            else: alldata = alldata.replace(ch,'')
    
    count = Counter(alldata)
    countSR = pd.DataFrame(count.values(),count.keys(), columns=[file])
    countSR = countSR/countSR.sum()
    countSR = countSR.T
    featureDF = pd.concat([featureDF, countSR], axis=1)
    targetSR.append(label)

targetDF = pd.DataFrame(targetSR, index=[featureDF.index])

cnt = 0
for i in featureDF.index:
    if featureDF.loc[i].sum() <= 0.0005:
        cnt+=1
        featureDF.drop(i, inplace=True)

featureDF.fillna(0)

,t,h,e,m,a,i,n,r,y,f,...,j,f,v,q,x,z,á,í,ó,é
en-1.txt,0.080522,0.043743,0.105332,0.026333,0.075952,0.073993,0.077476,0.077693,0.020022,0.015669,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.00000,0.000000,0.00000
en-2.txt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.00000,0.000000,0.00000
en-3.txt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.00000,0.000000,0.00000
en-4.txt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.00000,0.000000,0.00000
en-5.txt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.00000,0.000000,0.00000
fr-10.txt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.00000,0.000000,0.00000
fr-6.txt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.00000,0.000000,0.00000
fr-7.txt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.00000,0.000000,0.00000
fr-8.txt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.00000,0.000000,0.00000
fr-9.txt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0000,0.00000,0.000000,0.00000


In [191]:
targetDF = pd.get_dummies(targetDF)
targetDF.columns = ['en','fr','id','tl']
targetDF

,en,fr,id,tl
en-1.txt,True,False,False,False
en-2.txt,True,False,False,False
en-3.txt,True,False,False,False
en-4.txt,True,False,False,False
en-5.txt,True,False,False,False
fr-10.txt,False,True,False,False
fr-6.txt,False,True,False,False
fr-7.txt,False,True,False,False
fr-8.txt,False,True,False,False
fr-9.txt,False,True,False,False


In [192]:
type(featureDF)

pandas.core.frame.DataFrame

In [193]:
class IrisMCFmodel(nn.Module):
    def __init__(self):
        super().__init__()

        self.in_layer = nn.Linear(20, 50)
        self.hid_layer = nn.Linear(50, 30)
        self.out_layer = nn.Linear(30, 4)
    
    def forward(self,input):
        y = self.in_layer(input)
        y = F.relu(y)

        y = self.hid_layer(y)
        y = F.relu(y)

        return F. softmax(y)
    
class IrisDataset(Dataset):
    def __init__(self, featureDF, targetDF):
        self.featureDF=featureDF 
        self.targetDF=targetDF
        self.n_rows=featureDF.shape[0]
        self.n_features=featureDF.shape[1]
    
    def __len__(self):
        return self.n_rows
    
    def __getitem__(self, index):
        # 텐서화
        featureTS=torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS=torch.FloatTensor(self.targetDF.iloc[index].values) 
        
        # 피쳐와 타겟 반환
        return featureTS, targetTS


model = IrisMCFmodel()
irisDS=IrisDataset(featureDF, targetSR)
irisDL = DataLoader(irisDS)


In [194]:
EPOCH = 100
BATCH_SIZE = 1
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

In [195]:
featureDF.shape, targetDF.shape

((20, 701), (20, 4))

In [196]:
X_train, X_test, y_train, y_test =train_test_split(featureDF, targetDF, random_state=1)
X_train, X_val, y_train, y_val =train_test_split(X_train, y_train, random_state=1)

# 최적화 인스턴스 => W,b 텐서 즉, model.parameters() 전달
optimizer=optim.Adam(model.parameters(), lr=LR)

trainDS=IrisDataset(X_train, y_train)
valDS=IrisDataset(X_val, y_val)
testDS=IrisDataset(X_test, y_test)

# - 학습용 데이터로더 인스턴스
trainDL=DataLoader(trainDS, batch_size=BATCH_SIZE)

# 손실함수 인스턴스 => 분류 => 다중분류 CrossEntropyLoss
#                            예측값은 선형식 결과 값 전달 ==> AF 처리 X
crossLoss=nn.CrossEntropyLoss()

In [205]:
LOSS_HISTORY, SCORE_HISTORY=[[],[]], [[],[]]
CNT=len(trainDL)
print(f'BATCH_CNT => {CNT}')
breakCNT = 0
for epoch in range(EPOCH):
    # 학습 모드로 모델 설정
    model.train()

    
    # 배치 크기 만큼 데이터 로딩해서 학습 진행
    loss_total, score_total=0,0
    for featureTS, targetTS in trainDL:
        # 학습 진행
        print(featureTS.shape)
        pre_y=model(featureTS)
        
        # 손실 계산 : nn.CrossEntropyLoss 요구사항 : 정답/타겟은 0D 또는 1D,  타입은 long
        loss = crossLoss(pre_y, targetTS.reshape(-1).long())
        loss_total += loss.item()
        
        # 성능평가 계산
        score=MulticlassF1Score(num_classes=3)(pre_y, targetTS.reshape(-1))
        score_total += score.item()
        
        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # 에포크 당 검증기능
    # 모델 검증 모드 설정
    model.eval()
    with torch.no_grad():
        # 검증 데이터셋
        val_featrueTS=torch.FloatTensor(valDS.featureDF.values)
        val_targetTS=torch.FloatTensor(valDS.targetDF.values)
        
        # 추론/평가
        pre_val=model(val_featrueTS)
        # print(pre_val.shape, val_targetTS.reshape(-1).shape)
        
        # 손실
        loss_val=crossLoss(pre_val, val_targetTS.reshape(-1).long())
        score_val=MulticlassF1Score(num_classes=3)(pre_val, val_targetTS.reshape(-1))

    # 에포크 당 손실값과 성능평가값 저장    
    LOSS_HISTORY[0].append(loss_total/CNT)
    SCORE_HISTORY[0].append(score_total/CNT)
    
    LOSS_HISTORY[1].append(loss_val)
    SCORE_HISTORY[1].append(score_val)


    print(f'[{epoch}/{EPOCH}]\n- [TRAIN] LOSS : {LOSS_HISTORY[0][-1]} SCORE : {SCORE_HISTORY[0][-1]}')
    print(f'- [VALID] LOSS : {LOSS_HISTORY[1][-1]} SCORE : {SCORE_HISTORY[1][-1]} {breakCNT}')

BATCH_CNT => 11
torch.Size([1, 701])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x701 and 20x50)